# AI Travel Planner

This notebook uses smolagents and Gemini to provide live web results for flights, hotels, and itinerary recommendations.
The user enters a single prompt and all agents are called automatically.

## Install dependencies

In [2]:
!pip install smolagents google-generativeai duckduckgo-search ddgs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.4/145.4 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 101.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 90.4 MB/s eta 0:00:00
  Attempting uninstall: lxml
    Found existing installation: lxml 5.4.0
    Uninstalling lxml-5.4.0:
      Successfully uninstalled lxml-5.4.0


## Setup Gemini API

In [3]:
import google.generativeai as genai
from getpass import getpass

API_KEY = getpass('Enter your Gemini API key: ')
genai.configure(api_key=API_KEY)

Enter your Gemini API key: ··········


## Load Gemini Model

In [4]:
from smolagents import OpenAIServerModel

model = OpenAIServerModel(
    model_id='gemini-2.0-flash',
    api_base='https://generativelanguage.googleapis.com/v1beta',
    api_key=API_KEY
)

## Define Agents using Smolagents

In [5]:
from smolagents import DuckDuckGoSearchTool, tool

# Use DuckDuckGoSearchTool to retrieve live web info
search_tool = DuckDuckGoSearchTool()

# Flights agent
@tool
def flights_agent(origin: str, destination: str, date: str) -> str:
    """Searches for flights.

    Args:
        origin: The origin city.
        destination: The destination city.
        date: The travel date in YYYY-MM-DD format.
    """
    query = f'flights from {origin} to {destination} on {date} site:skyscanner.com OR site:expedia.com'
    results = search_tool(query)
    return f'Flight search results:\n{results}'

# Hotels agent
@tool
def hotels_agent(city: str, nights: int) -> str:
    """Searches for hotels.

    Args:
        city: The city to search for hotels.
        nights: The number of nights for the stay.
    """
    query = f'hotels in {city} for {nights} nights site:booking.com OR site:expedia.com'
    results = search_tool(query)
    return f'Hotel search results:\n{results}'

# Roadtrip agent
@tool
def itinerary_agent(city: str, nights: int) -> str:
    """Suggests itinerary items.

    Args:
        city: The city for the itinerary.
        nights: The number of nights for the trip.
    """
    query = f'top things to do in {city} {nights} days site:tripadvisor.com'
    results = search_tool(query)
    return f'Itinerary suggestions:\n{results}'

## Travel Planner

In [6]:
# Ask user for travel details in one prompt
user_input = input('Enter your travel plan (origin, destination, date (YYYY-MM-DD), nights): ')

# Gemini parses the user input into structured info
parse_prompt = f"""
You are a travel assistant.
Extract the following information from the user's input:
- origin city
- destination city
- travel date (YYYY-MM-DD)
- number of nights

User input: {user_input}
Provide the answer in the format:
origin=<city>;destination=<city>;date=<YYYY-MM-DD>;nights=<number>
"""

# Use the configured genai model to parse the input
parsed = genai.GenerativeModel('gemini-2.0-flash').generate_content(parse_prompt)
parsed_text = parsed.text.strip()

# Convert parsed info to dict
info = dict(item.split('=') for item in parsed_text.replace(' ','').split(';'))

# Run all agents
flights_results = flights_agent(info['origin'], info['destination'], info['date'])
hotels_results = hotels_agent(info['destination'], int(info['nights']))
itinerary_results = itinerary_agent(info['destination'], int(info['nights']))

# Combine results
final_plan = f"--- TRAVEL PLAN ---\n\n{flights_results}\n\n{hotels_results}\n\n{itinerary_results}"

print(final_plan)

Enter your travel plan (origin, destination, date, nights): casalblanca, tokyo, 2025-10-10, 7
--- TRAVEL PLAN ---

Flight search results:
## Search Results

[$183 Cheap Flights to Tokyo - Expedia](https://www.expedia.com/Cheap-Flights-To-Tokyo.d179900.Travel-Guide-Flights)
Looking for cheap flights to Tokyo ? Many airlines offer no change fee on selected flights and book now to earn your airline miles on top of our rewards! Find great 2025 Tokyo flight deals now!

[Cheap Flights, Plane Tickets & Airline Deals - Expedia](https://www.expedia.com/Flights)
Sep 19, 2010 · Book cheap flights with Expedia and select from thousands of cheap airline tickets. Earn your airline miles on top of our rewards!

[$258 Cheap Flights & Airfare to Japan - Expedia.com](https://www.expedia.com/Destinations-In-Japan.d89.Flight-Destinations)
Jan 14, 2015 · Find cheap return or one-way flights to Japan. Book & compare flight deals to Japan and save now! Get great flight deals to Japan for 2025 .

[The best da